# Example of RNA 5' Expansion Mapping and Annotation Pipeline (REMAP)

Jianheng Fox Liu, 2025.06

In [1]:
name = "test_run"

In [2]:
# Environments
cutadapt = "/home/fox/Software/bin/cutadapt"
umitools = "/home/fox/Software/bin/umi_tools"
hisat2 = "/home/fox/Software/hisat2/2.2.1/hisat2"
bowtie2 = "/home/fox/Software/bowtie2/2.4.2/bowtie2"
samtools = "/home/fox/Software/samtools/1.16/bin/samtools"
python = "/home/fox/Software/bin/python"

In [3]:
# Metadata
ncRNA_bt2 = "/home/fox/Database/Overrepresent_RNA_removal/20240127_silva_snRNA_snoRNA"  # this is for bowtie2 mapping for snRNA, snoRNA, and rRNA.
hisat2_index = "/home/fox/Database/hisat2/GRCh38_Gencode_v45_nochr_UCSC_SNP_v151/GRCh38.primary_assembly.genome.nochr.snp" # hisat2 index, with or withou SNP annotations
snp_db = "/home/fox/Database/dbSNP/UCSC_SNP151_COMMON/snp151Common.bed.gz" # dbSNP annotations, in BED format, optional
fasta = "/home/fox/Database/Genome/Human/Gencode/v45/GRCh38.primary_assembly.genome.nochr.fa" # fasta for the genomic sequence

In [4]:
# Scripts <-- change the directories
clean_bam = "clean_reads_in_BAM_by_UMI.py"
REMAP = "REMAP.v1.py"

## 1. Cutadapt and UMI-tools to remove artificial sequences

In [5]:
cmd_cutadapt = "{cutadapt} -j 4 -m 30 -q 20 -e 0.25 --max-n 0 -a AGATCGGAAGAGCACACGTC -A ATATN{{11}}AGATCGGAAGAGCGTCGTG -o read1.cutadapt.fastq.gz -p read2.cutadapt.fastq.gz {read1} {read2}".format(cutadapt=cutadapt, read1="test_read1.fastq.gz", read2="test_read2.fastq.gz")

In [6]:
!$cmd_cutadapt

This is cutadapt 4.1 with Python 3.9.14
Command line parameters: -j 4 -m 30 -q 20 -e 0.25 --max-n 0 -a AGATCGGAAGAGCACACGTC -A ATATN{11}AGATCGGAAGAGCGTCGTG -o read1.cutadapt.fastq.gz -p read2.cutadapt.fastq.gz test_read1.fastq.gz test_read2.fastq.gz
Processing paired-end reads on 4 cores ...
Done           00:00:01       100,000 reads @  14.6 µs/read;   4.12 M reads/minute
Finished in 1.47 s (15 µs/read; 4.07 M reads/minute).

=== Summary ===

Total read pairs processed:            100,000
  Read 1 with adapter:                  77,818 (77.8%)
  Read 2 with adapter:                  63,913 (63.9%)

== Read fate breakdown ==
Pairs that were too short:              34,402 (34.4%)
Pairs with too many N:                   1,102 (1.1%)
Pairs written (passing filters):        64,496 (64.5%)

Total basepairs processed:    20,200,000 bp
  Read 1:    10,100,000 bp
  Read 2:    10,100,000 bp
Quality-trimmed:                 413,879 bp (2.0%)
  Read 1:        92,500 bp
  Read 2:       321,379 bp


In [7]:
# If you have UMIs
cmd_umitools = "{umitools} extract -I read1.cutadapt.fastq.gz -S read1.umi.fastq --read2-in=read2.cutadapt.fastq.gz --read2-out=read2.umi.fastq -p NNNNNNNNNNNNNNN --log=umi_extract.log".format(umitools=umitools)
!$cmd_umitools

## 2. Bowtie2 to exclude unwanted transcripts (optional)

**Note**: In ReCappable-seq, many of the reads are from snRNA, snoRNA, and 7SK. Removing them first will make the analysis faster.

In [8]:
cmd_bowtie2_ncRNA = "{bowtie2} -p 6 --mm --fr -1 read1.umi.fastq -2 read2.umi.fastq --un-conc non_rRNA.%.fastq -x {ncRNA_2_bt2} --fast-local --dovetail |  {samtools} view -bS -F 4 -@ 6 -o ncRNA.bam".format(bowtie2=bowtie2, ncRNA_2_bt2=ncRNA_bt2, samtools=samtools)
!$cmd_bowtie2_ncRNA

64496 reads; of these:
  64496 (100.00%) were paired; of these:
    33413 (51.81%) aligned concordantly 0 times
    3205 (4.97%) aligned concordantly exactly 1 time
    27878 (43.22%) aligned concordantly >1 times
    ----
    33413 pairs aligned concordantly 0 times; of these:
      0 (0.00%) aligned discordantly 1 time
    ----
    33413 pairs aligned 0 times concordantly or discordantly; of these:
      66826 mates make up the pairs; of these:
        66666 (99.76%) aligned 0 times
        25 (0.04%) aligned exactly 1 time
        135 (0.20%) aligned >1 times
48.32% overall alignment rate


## 3. Mapping to the reference genome via HISAT2

**Note:** HISAT2 has some features that allows better performance in finding the expansions wiht the default settings. STAR is not recommended.

In [9]:
cmd_hisat2 = "{hisat2} -x {hisat2_index} --no-discordant --no-mixed --fr --rna-strandness FR -p 6 -1 non_rRNA.1.fastq -2 non_rRNA.2.fastq | {samtools} view -bS -F 4 -@ 6 -o hisat2.bam".format(hisat2=hisat2, hisat2_index=hisat2_index, samtools=samtools)
!$cmd_hisat2

33413 reads; of these:
  33413 (100.00%) were paired; of these:
    2880 (8.62%) aligned concordantly 0 times
    27813 (83.24%) aligned concordantly exactly 1 time
    2720 (8.14%) aligned concordantly >1 times
91.38% overall alignment rate


In [10]:
# Optional. This script will check the UMI sequence, which should end with ATAT in my case. Skip if you don't need this
# Ignore the warnings
!$python $clean_bam hisat2.bam hisat2.clean.bam

[E::idx_find_and_load] Could not retrieve index file for 'hisat2.bam'


In [11]:
!$samtools sort -m 4G -@ 4 -o hisat2.clean.sorted.bam hisat2.clean.bam
!$samtools index hisat2.clean.sorted.bam

[bam_sort_core] merging from 0 files and 4 in-memory blocks...


## 4. Deduplication with UMIs (optional)

In [12]:
cmd_umitools_dedup = "{umitools} dedup --paired --chimeric-pairs=discard --unpaired-reads=discard --stdin=hisat2.clean.sorted.bam --log=umi.logs --method=unique > {name}.bam ".format(umitools=umitools, name=name)
!$cmd_umitools_dedup

In [13]:
cmd_samtools_index = "{samtools} index {name}.bam".format(samtools=samtools, name=name)
!$cmd_samtools_index

## 5. Fix the BAM file and find out RNA 5' expansions

In [14]:
cmd_reannotate_softclippings = "{python} {REMAP} -b {name}.bam -s {snp} -f {ref_genome_nochr} -o {name}.expansions.csv".format(python=python, name=name, ref_genome_nochr=fasta, snp=snp_db, REMAP=REMAP)
!$cmd_reannotate_softclippings

[2025-06-27 13:33:44] Loading FASTA...
[2025-06-27 13:34:01] FASTA loaded.
[2025-06-27 13:34:01] Reads in BAM file (read1 + read2 + seconadary + etc.): 70995
[2025-06-27 13:34:01] Analyzing BAM file with 4 processors.
[2025-06-27 13:34:01] Merging temp BAM files...
[2025-06-27 13:34:02] BAM file with corrected 5' annotation: test_run.softclip_corrected.bam
[2025-06-27 13:34:02] Reading corrected BAM file...
[2025-06-27 13:34:02] Checking SNPs...
[2025-06-27 13:34:02] Total reads (Read1 only. Expected 1/2 as paired-end read counts): 29087


### Here is the example CSV output:

chr - chromosome ID

pos - 1-based location of the TSS

strand - strand 

ref_base - reference base (with strandness)

softclip_bases - the expanded bases

Count - number of reads

Softclip_size - the length of softclipping

COMMENT - the type of expansion:

* M - no expansion

* +1 EXPANDED, expansion at the first nucleotide, e.g, AAAG -> AAAAG

* +2 EXPANDED, expansion at the second nucleotide, e.g., ATTTG -> ATTTTTTG

* +3 EXPANDED, expansion at the third nucleotide, e.g., TCAAAAG -> TCAAAAAAG

* +4 EXPANDED, expansion at the forth nucleotide

* Reanneal_xx_yy, the nascent RNA dissociated and reannealed upstream before elongation. The xx is the inserted nucleotides, and yy is the sequence gapped between the reannealed site and the TSS

* SNP

* UNKNOWN, unclassified, many of them are sequencing/mapping errors, some of them are complicated expansion cases



In [15]:
!head test_run.expansions.csv

,chr,pos,strand,ref_base,softclip_bases,Count,Softclip_size,COMMENT
0,1,199838,-,A,A,1,0,M
1,1,778632,-,G,G,2,0,M
2,1,959258,-,A,A,1,0,M
3,1,959267,-,G,G,1,0,M
4,1,959268,-,A,A,1,0,M
5,1,966976,-,A,A,1,0,M
6,1,1000123,-,A,A,2,0,M
7,1,1000339,+,G,G,1,0,M
8,1,1000815,-,A,A,1,0,M


**Note:** The output table shows all different events of expansions per TSS. To format the table(s), please refer to **Example_statistics_pipeline** for more details.